In [1]:
# import libraries
from urllib.request import urlopen
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
# import urllib.request
import re
import pandas as pd
import os
import pandas as pd
from pymongo import MongoClient
import json
from datetime import datetime
#import keyring

import pprint
from datetime import datetime, date
import calendar

# EXTRACTING DATA FROM WEBSITE

In [72]:
# Specify the url
quote_page="https://open-enernoc-data.s3.amazonaws.com/anon/csv/index.html"
page =urlopen(quote_page)
# print(page)
soup = BeautifulSoup(page, "html.parser")
name_box = soup.find_all("a")
# print(len(name_box))

list_csv=[elem.text for elem in name_box if '.csv' in elem.text]
link="https://open-enernoc-data.s3.amazonaws.com/anon/csv/"
# print("number of files to retrieve",len(list_csv))

#### ALL CSV FILES DONE

In [ ]:
for file in list_csv:
     urlretrieve(link+file,"data_csv/"+file)

#### METADATA DONE

In [ ]:
metadata_link="https://open-enernoc-data.s3.amazonaws.com/anon/meta/all_sites.csv"
urlretrieve(metadata_link,"data_csv/metadata.csv")

# CONCAT DATA BEFORE INTEGRATION IN DB

In [2]:
data=pd.read_csv("all_data.csv")
data=data.drop("Unnamed: 0",axis=1)
data.head()

,timestamp,dttm_utc,value,estimated,anomaly,SITE_ID
0,1325376600,2012-01-01 00:10:00,105.7895,0,NaN,10
1,1325376900,2012-01-01 00:15:00,104.7318,0,NaN,10
2,1325377200,2012-01-01 00:20:00,102.2920,0,NaN,10
3,1325377500,2012-01-01 00:25:00,102.7851,0,NaN,10
4,1325377800,2012-01-01 00:30:00,101.6106,0,NaN,10


In [3]:
# meta_path = "C:\\Users\\sof_sur\\Documents\\ANACONDA\\ANACONDA3\\PythonScripts_v3\\CES\\Module5-BD\\meta\\all_sites.csv"
meta_path = "data_csv/metadata.csv"
meta_data=pd.read_csv(meta_path)
meta_data.head()

,SITE_ID,INDUSTRY,SUB_INDUSTRY,SQ_FT,LAT,LNG,TIME_ZONE,TZ_OFFSET
0,6,Commercial Property,Shopping Center/Shopping Mall,161532,34.783001,-106.895250,America/Denver,-06:00
1,8,Commercial Property,Shopping Center/Shopping Mall,823966,40.320247,-76.404942,America/New_York,-04:00
2,9,Commercial Property,Corporate Office,169420,40.946751,-74.742087,America/New_York,-04:00
3,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00
4,12,Commercial Property,Business Services,179665,39.694541,-74.899166,America/New_York,-04:00


### Merge between detailed_data and meta_data

In [4]:
df_final = pd.merge(data,meta_data, how='left', on = "SITE_ID")

# IMPORT IN MONGODB

In [5]:
import datetime

def date_type(json_dict):
    for (key, value) in json_dict.items():
        try:
            json_dict[key] = datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S")
        except:
            pass
    return json_dict

def mongoimport(csv_path, meta_path, db_name, coll_name, link, db_url='localhost', db_port=27017):
    """ Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = MongoClient(db_url, db_port)
    
    db = client[db_name]
    coll = db[coll_name]
    
    data = pd.read_csv(csv_path)
    meta= pd.read_csv(meta_path)
    ## ajout de l'index de l'entreprise pour les datas de conso qui est le nom du fichier csv 
    namefile= str(csv_path).replace(link,"")
    namefile=namefile.replace(".csv","")
    
    meta_to_integrate=meta[meta['SITE_ID']==int(namefile)]
#     c'est ici que je fais du "nested" or "embedded" document
#     la solution choisie est de faire un document embedded, ie dans chaque ligne du meta_données, correspondant à un site, 
#     on intégre la timeserie de consommation dans un champ "nested" ld_data
    meta_to_integrate['ld_data']=data.to_json(orient='records')
    meta_to_integrate['ld_data']=meta_to_integrate['ld_data'].apply(lambda x: json.loads(x))
    
#     date_type permet d'intégrer les dates entant que format "date" dans mongodb (ISODate)
    payload = json.loads(meta_to_integrate.to_json(orient='records'),object_hook=date_type)
    coll.insert(payload)
    print(set(meta_to_integrate.SITE_ID),'\t',coll.count())
    return coll.count()

In [6]:
def creationBD(link,db_name,coll_name):
    for element in os.listdir(link):
        print("element",element)
        if element!='metadata.csv':
            csv_path = link + element 
#             meta_path = "C:\\Users\\sof_sur\\Documents\\ANACONDA\\ANACONDA3\\PythonScripts_v3\\CES\\Module5-BD\\meta\\all_sites.csv"
            mongoimport(csv_path,meta_path, db_name, coll_name,link, db_url='localhost', db_port=27017)

### INTEGRATE DATA IN DB ==> file by file and creating SITE_ID and merging meta data

In [41]:
%%time
# csv_path = "data_csv/"
csv_path = "data_test_csv/"
meta_path = "data_csv/metadata.csv"
db_name='telecom_ds'
coll_name="coll_enernoc"
# coll_name="data_two_sites"
    
creationBD(csv_path,db_name,coll_name)
# Wall time: 5min 4s

element 10.csv


C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead.

{10} 	 1
element 41.csv
{41} 	 2
element 6.csv
{6} 	 3
element 8.csv
{8} 	 4
element 9.csv
{9} 	 5
Wall time: 14.2 s


In [28]:
mongo_client = MongoClient('mongodb://localhost:27017')
db = mongo_client.telecom_ds
coll_enernoc=db.coll_enernoc

### Working with dataframe to check with mongoDB results

In [8]:
df_final.head(3)

,timestamp,dttm_utc,value,estimated,anomaly,SITE_ID,INDUSTRY,SUB_INDUSTRY,SQ_FT,LAT,LNG,TIME_ZONE,TZ_OFFSET
0,1325376600,2012-01-01 00:10:00,105.7895,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00
1,1325376900,2012-01-01 00:15:00,104.7318,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00
2,1325377200,2012-01-01 00:20:00,102.2920,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00


In [9]:
%%time
df_final['dttm_utc']=df_final['dttm_utc'].apply(lambda row: pd.Timestamp(row))
df_final['dttm_utc_2']=df_final['timestamp'].apply(lambda row: pd.Timestamp(row,unit='s'))

Wall time: 49.1 s


In [10]:
df_final[['timestamp','dttm_utc','dttm_utc_2']].tail()

,timestamp,dttm_utc,dttm_utc_2
10531283,1356997200,2012-12-31 23:40:00,2012-12-31 23:40:00
10531284,1356997500,2012-12-31 23:45:00,2012-12-31 23:45:00
10531285,1356997800,2012-12-31 23:50:00,2012-12-31 23:50:00
10531286,1356998100,2012-12-31 23:55:00,2012-12-31 23:55:00
10531287,1356998400,2013-01-01 00:00:00,2013-01-01 00:00:00


In [11]:
%%time
df_sample=df_final.resample('5T',on='dttm_utc')[['value']].sum()

# To check if the timestamp vs dttm_utc are good:
df_sample=df_sample.reset_index()
df_sample['timestamp']=df_sample['dttm_utc'].apply(lambda x: int(calendar.timegm(x.timetuple()))) #UTC Zone
# df_sample['timestamp']=df_sample['dttm_utc'].apply(lambda x: int(datetime.datetime.utcfromtimestamp(x)))
df_sample['timestamp_2']=df_sample['timestamp'].apply(lambda x: pd.Timestamp(x,unit='s'))
print(df_sample.shape)
df_sample.tail()

(105408, 4)
Wall time: 10.2 s


In [12]:
period=5*60
df_resample_by_timestamp=df_final.groupby(df_final.timestamp // period)[['value']].sum()
df_resample_by_timestamp.tail()

,value
timestamp,
4523324,3868.5148
4523325,3823.1735
4523326,3850.0199
4523327,3847.1291
4523328,3821.4506


In [13]:
df_sort=df_final.sort_values(by='timestamp',ascending=False)
df_sort.head(3)

,timestamp,dttm_utc,value,estimated,anomaly,SITE_ID,INDUSTRY,SUB_INDUSTRY,SQ_FT,LAT,LNG,TIME_ZONE,TZ_OFFSET,dttm_utc_2
10531287,1356998400,2013-01-01,3.4221,0,NaN,99,Education,Primary/Secondary School,79503,36.575496,-86.227413,America/Chicago,-05:00,2013-01-01
3790379,1356998400,2013-01-01,8.5378,0,NaN,32,Commercial Property,Shopping Center/Shopping Mall,880941,34.373903,-87.046518,America/Chicago,-05:00,2013-01-01
8005461,1356998400,2013-01-01,21.5030,0,NaN,703,Light Industrial,Food Processing,117651,44.471774,-75.617895,America/New_York,-04:00,2013-01-01


### Requete 1 : grouper par timestamp 
•	Calculer la somme LD pour les 100 sites (intervalle de timestamp: 5 minutes)
- timestamp min = 1 janv 2012 à minuit 5 = 
- timestamp max = 1 janv 2013 à minuit = 
- max-min = 31622100
- 5 minutes = 300 en timsetamp. 
- il doit donc y avoir 31622100 / 300 = 105407 résultats

In [43]:
%%time
### pour l'ensemble des sites 
# Wall time: 42 s
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "ld_data.value" : 1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"},
                    {"$mod": [{"$toLong":"$ld_data.timestamp"}, 300]},
                ]
            }
        }
    },

    {   
        "$group": {"_id": {"time": "$time"}, "total":{"$sum":"$ld_data.value"}}
    },
    {
        "$sort":{"_id.time":-1}
    }
]

requete1 = list(coll_enernoc.aggregate(pipeline))
pprint.pprint(len(requete1))

105408
Wall time: 42 s


In [44]:
requete1[:5]

[{'_id': {'time': 1356998400}, 'count': 3821.4506},
 {'_id': {'time': 1356998100}, 'count': 3847.1291},
 {'_id': {'time': 1356997800}, 'count': 3850.0199},
 {'_id': {'time': 1356997500}, 'count': 3823.1735},
 {'_id': {'time': 1356997200}, 'count': 3868.5148}]

##### Checking using dataframe

In [42]:
df_sample2=df_sort.groupby('timestamp')[['value']].sum().sort_values(by="timestamp",ascending=False)
df_sample2.head()

,value
timestamp,
1356998400,3821.4506
1356998100,3847.1291
1356997800,3850.0199
1356997500,3823.1735
1356997200,3868.5148


##### Storing requete1 in a dataframe to check results with df_sample2 ==> ALL GOOD

In [39]:
df_req1=pd.DataFrame(requete1)
df_req1['_id']=df_req1['_id'].apply(lambda x:str(x)[-11:-1])
df_req1=df_req1.sort_values(by='_id',ascending=False)
df_req1.head()

,_id,count
7184,1356998400,3821.4506
60683,1356998100,3847.1291
56983,1356997800,3850.0199
20089,1356997500,3823.1735
103067,1356997200,3868.5148


##### Requete1 bis ==> par site ==> très très longue

In [ ]:
%%time
### Pour chaque site , good 
# Wall time: 27min
pipeline = [
     {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
#             "_id":"$_id",
            "ld_data.value" : 1,
            "SITE_ID":1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"},
                    {"$mod": [{"$toLong":"$ld_data.timestamp"}, 300]}
                ]
            }
        }
    },
    {   
        "$group": {"_id": {"SITE_ID":"$SITE_ID","time": "$time"}, "total":{"$sum":'$ld_data.value'}}
    }
]
    
requete1_bis = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

In [ ]:
pprint.pprint(len(requete1_bis))
pprint.pprint(requete1_bis[0])
pprint.pprint(requete1_bis[1])
pprint.pprint(requete1_bis[2])
pprint.pprint(requete1_bis[3])
pprint.pprint(requete1_bis[-2])
pprint.pprint(requete1_bis[-1])

### Requête 2 
•	Calculer le LD moyen par secteur d'activité (intervalle de timestamp: 5 minutes) 

2 solutions : 
- soit on fait une seule collection avec tout dedans, 
- soit on fait 2 collections mais avec une jointure. 


Dans un cas le requetage est facile, dans le 2eme il n'y a pas de redondance pour les sites. 

Jointure pas optimale pour mongoDB

==> du coup la solution choisie est de faire un document embedded, ie dans chaque ligne du meta_données, correspondant à un site, on intégre la timeserie de consommation dans un champ "nested" ld_data

In [84]:
%%time
# Wall time: 1min 50s
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "ld_data.value" : 1,
            "SUB_INDUSTRY":1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"},
                    {"$mod": [{"$toLong":"$ld_data.timestamp"}, 300]}
                ]
            }
        }
    },
    {   
        "$group": {"_id": {"SUB_INDUSTRY":"$SUB_INDUSTRY","time": "$time"}, "average":{"$avg":'$ld_data.value'}}
    },
    {
        "$sort":{"SUB_INDUSTRY":1}
    }
]
    
requete2 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

Wall time: 1min 37s


In [85]:
requete2[:5]

[{'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'time': 1325376300},
  'avergae': 7.3969},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'time': 1325376600},
  'avergae': 6.8279},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'time': 1325376900},
  'avergae': 7.3969},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'time': 1325377200},
  'avergae': 7.3969},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'time': 1325377500},
  'avergae': 7.9659}]

##### Checking using datarame

In [82]:
df_sample3=df_sort.groupby(['SUB_INDUSTRY','timestamp'])[['value']].mean()#.sort_values(by="timestamp",ascending=False)
df_sample3.tail()

value
SUB_INDUSTRY                  timestamp            
Shopping Center/Shopping Mall 1356997200  74.055479
                              1356997500  72.564250
                              1356997800  72.915443
                              1356998100  74.306386
                              1356998400  72.163643

In [83]:
df_sample3.head()

value
SUB_INDUSTRY            timestamp         
Bank/Financial Services 1325376300  7.3969
                        1325376600  6.8279
                        1325376900  7.3969
                        1325377200  7.3969
                        1325377500  7.9659

### Requête 3
•	Calculer le LD total pour les 100 sites (intervalle de timestamp: une semaine)

In [86]:
%%time
# Wall time: 30.6 s
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
#             "_id":"$_id",
            "ld_data.value" : 1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"},
                    {"$mod": [{"$toLong":"$ld_data.timestamp"}, 604800]}
                ]
            }
        }
    },
    {   
        "$group": {"_id": {"time": "$time"}, "total":{"$sum":'$ld_data.value'}}
    },
    {
        "$sort":{"_id.time":-1}
    }
]
    
requete3 = list(coll_enernoc.aggregate(pipeline))

Wall time: 40.1 s


In [88]:
requete3[:5]

[{'_id': {'time': 1356566400}, 'total': 5976001.0668},
 {'_id': {'time': 1355961600}, 'total': 7907839.6715},
 {'_id': {'time': 1355356800}, 'total': 9816839.8449},
 {'_id': {'time': 1354752000}, 'total': 9641062.3717},
 {'_id': {'time': 1354147200}, 'total': 9691423.5444}]

##### Checking using datarame

In [79]:
period=7*24*60*60
df_resample_1W=df_final.groupby(df_final.timestamp // period)[['value']].sum()
df_resample_1W.tail()

,value
timestamp,
2239,9.691424e+06
2240,9.641062e+06
2241,9.816840e+06
2242,7.907840e+06
2243,5.976001e+06


In [13]:
#==> with this dataframe don't have the same result than df_resample_1W
# df_sample4=df_final.resample('10080T',on='dttm_utc')[['value']].sum()
df_sample4=df_final.resample('1W',on='dttm_utc')[['value']].sum()
df_sample4=df_sample4.reset_index()
df_sample4['timestamp']=df_sample4['dttm_utc'].apply(lambda x: int(calendar.timegm(x.timetuple()))) #UTC Zone
df_sample4.tail() 

,dttm_utc,value,timestamp
49,2012-12-09,9.672678e+06,1355011200
50,2012-12-16,9.768154e+06,1355616000
51,2012-12-23,9.739731e+06,1356220800
52,2012-12-30,7.476538e+06,1356825600
53,2013-01-06,1.181829e+06,1357430400


In [83]:
%%time
### Pour chaque site , good 
# Wall time: 1min 17s
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
#             "_id":"$_id",
            "ld_data.value" : 1,
            "SITE_ID":1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"}, {"$mod": [{"$toLong":"$ld_data.timestamp"}, 604800]}
                ]
            }
        }
    },
    {   
        "$group": {"_id": {"SITE_ID":"$SITE_ID","time": "$time"}, "total":{"$sum":'$ld_data.value'}}
    }
]
    
requete3_bis = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

Wall time: 1min 15s


In [84]:
requete3_bis[:5]

[{'_id': {'SITE_ID': 363, 'time': 1341446400}, 'count': 30690.6274},
 {'_id': {'SITE_ID': 742, 'time': 1337817600}, 'count': 147622.2897},
 {'_id': {'SITE_ID': 109, 'time': 1338422400}, 'count': 18219.891},
 {'_id': {'SITE_ID': 111, 'time': 1338422400}, 'count': 17356.0675},
 {'_id': {'SITE_ID': 21, 'time': 1342656000}, 'count': 201065.0886}]

##### Checking using dataframe

In [14]:
period=7*24*60*60
df_resample_1W=df_final.groupby(["SITE_ID",df_final.timestamp // period])[['value']].sum()
df_resample_1W=df_resample_1W.reset_index()
df_resample_1W.query("SITE_ID==363 and timestamp==2218.0") #1341446400/period

,SITE_ID,timestamp,value
2940,363,2218,30690.6274


### Requête 4
•	Calculer le LD moyen par secteur d'activité (intervalle de timestamp: une semaine)

In [89]:
%%time
# Wall time: 1min 9s
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
#             "_id":"$_id",
            "ld_data.value" : 1,
            "SUB_INDUSTRY":1,
            "time": {
                "$subtract": [
                    {"$toLong":"$ld_data.timestamp"},
                    {"$mod": [{"$toLong":"$ld_data.timestamp"}, 604800]}
                ]
            }
        }
    },
    {   
        "$group": {"_id": {"SECTEUR ACTIVITE":"$SUB_INDUSTRY","time": "$time"}, "average":{"$avg":'$ld_data.value'}}
    }
]
    
requete4 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

Wall time: 43.4 s


In [91]:
requete4[:5]

[{'_id': {'SECTEUR ACTIVITE': 'Primary/Secondary School', 'time': 1343260800},
  'average': 11.76767047420635},
 {'_id': {'SECTEUR ACTIVITE': 'Grocer/Market', 'time': 1348704000},
  'average': 18.94598034722222},
 {'_id': {'SECTEUR ACTIVITE': 'Food Processing', 'time': 1343865600},
  'average': 79.45437374686716},
 {'_id': {'SECTEUR ACTIVITE': 'Commercial Real Estate', 'time': 1329350400},
  'average': 60.18675497271825},
 {'_id': {'SECTEUR ACTIVITE': 'Manufacturing', 'time': 1343260800},
  'average': 158.4643633829365}]

### Requete 5 
•	Classer les secteurs d'activité par intensité énergétique (rapport consommation d'électricité / surface): au total, puis par saison (hiver, printemps, été, automne)

In [87]:
%%time
###au total 
# Wall time: 29.7 s
pipeline = [
    {
        "$unwind":"$ld_data"
    },

    {   
        "$group": {"_id" : {"SECTEUR_ACTIVITE" : "$SUB_INDUSTRY","SITE_ID":"$SITE_ID"}, 
                   "conso_sum" : { "$sum" : "$ld_data.value"} ,
                   "sq_ft" : { "$avg" : "$SQ_FT"},
                   }
    },
        {   
        "$project": {
            "_id":"$_id",
            "conso_sum":1,
            "sq_ft":1
        }
        },
      {   
        "$group": {"_id" : {"SECTEUR_ACTIVITE" : "$_id.SECTEUR_ACTIVITE"}, 
                   "conso_sum" : { "$sum" : "$conso_sum"} ,
                   "sq_ft_sum" : { "$sum" : "$sq_ft"},
                   }
    },
    {   
        "$project": {
            "_id":0,
            "sub_industry":"$_id.SECTEUR_ACTIVITE",
            "intensity": { "$divide": ["$conso_sum", "$sq_ft_sum"] },
            "conso_energy":"$conso_sum",
            "sq_ft":"$sq_ft_sum"
        }
    },
    {
        "$sort":{"energy":-1}
    }
]

requete5_total = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))
pprint.pprint(len(requete5_total))

10
Wall time: 57.9 s


In [88]:
db.intensity_industry_total.remove()
db.intensity_industry_total.insert(requete5_total)

C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5e90768324387643ba7b1fba'),
 ObjectId('5e90768324387643ba7b1fbb'),
 ObjectId('5e90768324387643ba7b1fbc'),
 ObjectId('5e90768324387643ba7b1fbd'),
 ObjectId('5e90768324387643ba7b1fbe'),
 ObjectId('5e90768324387643ba7b1fbf'),
 ObjectId('5e90768324387643ba7b1fc0'),
 ObjectId('5e90768324387643ba7b1fc1'),
 ObjectId('5e90768324387643ba7b1fc2'),
 ObjectId('5e90768324387643ba7b1fc3')]

In [89]:
requete5_total

[{'sub_industry': 'Grocer/Market',
  'intensity': 45.836270624129774,
  'conso_energy': 47931538.2269,
  'sq_ft': 1045712.0},
 {'sub_industry': 'Primary/Secondary School',
  'intensity': 8.269535184876979,
  'conso_energy': 28868475.9669,
  'sq_ft': 3490943.0},
 {'sub_industry': 'Shopping Center/Shopping Mall',
  'intensity': 15.819280220499278,
  'conso_energy': 169593282.8815,
  'sq_ft': 10720670.0},
 {'sub_industry': 'Corporate Office',
  'intensity': 16.455987745377783,
  'conso_energy': 5940183.7204,
  'sq_ft': 360974.0},
 {'sub_industry': 'Manufacturing',
  'intensity': 42.96044179897954,
  'conso_energy': 68065492.9357,
  'sq_ft': 1584376.0},
 {'sub_industry': 'Other Light Industrial',
  'intensity': 161.65402389775494,
  'conso_energy': 1195269.8527,
  'sq_ft': 7394.0},
 {'sub_industry': 'Business Services',
  'intensity': 14.398076085365792,
  'conso_energy': 28531083.59,
  'sq_ft': 1981590.0},
 {'sub_industry': 'Bank/Financial Services',
  'intensity': 9.71404584930651,
  'co

##### Cheking using dataframe

In [14]:
df_sqft=df_final.groupby(['SUB_INDUSTRY','SITE_ID'])[['SQ_FT']].mean()
df_sqft=df_sqft.groupby("SUB_INDUSTRY")[['SQ_FT']].sum()

df_value=df_final.groupby('SUB_INDUSTRY')[['value']].sum()

df_ratio=pd.concat([df_value,df_sqft],axis=1)
df_ratio['ratio']=(df_ratio['value']/df_ratio['SQ_FT'])

df_ratio

,value,SQ_FT,ratio
SUB_INDUSTRY,,,
Bank/Financial Services,7.913518e+06,814647,9.714046
Business Services,2.853108e+07,1981590,14.398076
Commercial Real Estate,2.415139e+07,1385021,17.437560
Corporate Office,5.940184e+06,360974,16.455988
Food Processing,1.425997e+08,1133956,125.754189
Grocer/Market,4.793154e+07,1045712,45.836271
Manufacturing,6.806549e+07,1584376,42.960442
Other Light Industrial,1.195270e+06,7394,161.654024
Primary/Secondary School,2.886848e+07,3490943,8.269535


#### SEASONS

In [15]:
%%time
df_final['month']=df_final['dttm_utc'].apply(lambda x:x.month)
df_final['day']=df_final['dttm_utc'].apply(lambda x:x.day)
df_final['season']='other'
df_final.loc[((df_final['month']==9)&(df_final['day']>=22))|((df_final['month']==12) & (df_final['day']<=20))|((df_final['month']==10)|(df_final['month']==11)),'season']='autumn'
df_final.loc[((df_final['month']==12)&(df_final['day']>=21))|((df_final['month']==3) & (df_final['day']<=19))|((df_final['month']==1)|(df_final['month']==2)),'season']='winter'
df_final.loc[((df_final['month']==3)&(df_final['day']>=20))|((df_final['month']==6) & (df_final['day']<=19))|((df_final['month']==4)|(df_final['month']==5)),'season']='spring'
df_final.loc[((df_final['month']==6)&(df_final['day']>=20))|((df_final['month']==9) & (df_final['day']<=21))|((df_final['month']==7)|(df_final['month']==8)),'season']='summer'

# cond_summer=season_boundary_cond(6,20,9,21,7,8)
# cond_autumn=season_boundary_cond(9,22,12,20,10,11)
# cond_winter=season_boundary_cond(12,21,3,19,1,2)
# cond_spring=season_boundary_cond(3,20,6,19,4,5)

Wall time: 1min 27s


In [16]:
df_final.head()

,timestamp,dttm_utc,value,estimated,anomaly,SITE_ID,INDUSTRY,SUB_INDUSTRY,SQ_FT,LAT,LNG,TIME_ZONE,TZ_OFFSET,dttm_utc_2,month,day,season
0,1325376600,2012-01-01 00:10:00,105.7895,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00,2012-01-01 00:10:00,1,1,winter
1,1325376900,2012-01-01 00:15:00,104.7318,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00,2012-01-01 00:15:00,1,1,winter
2,1325377200,2012-01-01 00:20:00,102.2920,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00,2012-01-01 00:20:00,1,1,winter
3,1325377500,2012-01-01 00:25:00,102.7851,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00,2012-01-01 00:25:00,1,1,winter
4,1325377800,2012-01-01 00:30:00,101.6106,0,NaN,10,Commercial Property,Shopping Center/Shopping Mall,1029798,39.732504,-75.006861,America/New_York,-04:00,2012-01-01 00:30:00,1,1,winter


In [17]:
# df_final_test=df_final.query("SITE_ID in [6,8,9,10,41]")
# df_work=df_final_test.copy()
df_work=df_final.copy()

df_sqft=df_work.groupby(['SUB_INDUSTRY','SITE_ID'])[['SQ_FT']].mean()
df_sqft=df_sqft.groupby("SUB_INDUSTRY")[['SQ_FT']].sum()

df_value_season=df_work.groupby(['SUB_INDUSTRY','season'])[['value']].agg(['sum','count']).reset_index().set_index('SUB_INDUSTRY')
# df_value_season=df_work.groupby(['SUB_INDUSTRY','season'])[['value']].sum().reset_index().set_index('SUB_INDUSTRY')

df_ratio_season=df_value_season.join(df_sqft)
df_ratio_season['ratio']=(df_ratio_season[('value', 'sum')]/df_ratio_season['SQ_FT'])

df_ratio_season

C:\Users\Hanane\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(season, )","(value, sum)","(value, count)",SQ_FT,ratio
SUB_INDUSTRY,,,,,
Bank/Financial Services,autumn,1.998107e+06,51840,814647,2.452727
Bank/Financial Services,spring,1.977249e+06,52992,814647,2.427123
Bank/Financial Services,summer,2.123996e+06,54143,814647,2.607260
Bank/Financial Services,winter,1.814167e+06,48441,814647,2.226936
Business Services,autumn,6.723733e+06,77758,1981590,3.393100
Business Services,spring,6.942102e+06,79482,1981590,3.503299
Business Services,summer,7.840074e+06,81214,1981590,3.956456
Business Services,winter,7.025175e+06,77759,1981590,3.545221
Commercial Real Estate,autumn,5.623101e+06,103679,1385021,4.059940


In [18]:
# Printemps: 20 Mars au 19 Juin
# Ete:       20 Juin au 21 Septembre
# Automne:   22 Septembre au 20 Décembre
# Hiver:     21 Décembre au 19 Mars


def season_boundary_cond(m_start,d_start,m_end,d_end,m_in_1,m_in_2):
    """
    Example: for "autumn" 22 September ==> 20 December ==> season_boundary_cond(9,22,12,20,10,11)
    must return {"$or":[{"$and":[{"month_":{"$eq": 9}} ,{"day": {"$gte":22}}]},
                       {"$and":[{"month_":{"$eq": 12}} ,{"day": {"$lte":20}}]},
                        {"$or":[{"$eq":["$month_", 10]} ,{"$eq":["$month_", 11]} ] }
                 }
    """
    return {"$or":[{"$and":[{"$eq":["$month_", m_start]} ,{"$gte": ["$day",d_start]}]},
                       {"$and":[{"$eq":["$month_", m_end]} ,{"$lte": ["$day",d_end]}]},
                       {"$or":[{"$eq":["$month_", m_in_1]} ,{"$eq":["$month_", m_in_2]} ]}
                  ]
                 }
cond_summer=season_boundary_cond(6,20,9,21,7,8)
cond_autumn=season_boundary_cond(9,22,12,20,10,11)
cond_winter=season_boundary_cond(12,21,3,19,1,2)
cond_spring=season_boundary_cond(3,20,6,19,4,5)

# seasons_boundary_cond=[cond_summer,cond_autumn,cond_winter,cond_spring]

In [57]:
def query_season(coll,cond_summer,cond_cond_autumn,cond_winter,cond_spring):

    # order: 1)unwind, 2)project1, 3) group1: SITE_ID for sq_ft calculation and using condition season (sq_ft is summed as many times as we access to ld_data rows)
    # 4) project2: calculation sq_ft per sub industry/site_id 5)group2: calculate conso and sq_ft per sub industry
    # 6)project3: calculate intensity
    pipeline = [
        {
            "$unwind":"$ld_data"
        },
        {   
            "$project": {
    #             "_id":"$_id.SUB_INDUSTRY",
                "SUB_INDUSTRY":1,
                "ld_data.value" : 1,
                "SQ_FT":1,
                "SITE_ID":1,
                "month_":{ "$month": "$ld_data.dttm_utc" },
                "day":{ "$dayOfMonth": "$ld_data.dttm_utc" },
                
            }
        },
        {   
            "$group": {
                        "_id" : {"SECTEUR_ACTIVITE" : "$SUB_INDUSTRY", "SITE_ID":"$SITE_ID",
                
                        "season":{"$cond":[
                            cond_summer,"summer", {"$cond":[
                                cond_autumn,"autumn", {"$cond":[
                                    cond_winter,"winter", {"$cond":[
                                        cond_spring,"spring","other"
                                                            ]}
                                ]}
                            ]}
                        ]}
                    },
                    "conso_sum" : { "$sum" : "$ld_data.value"} ,
                    "count_row":{"$sum":1},
                    "sq_ft_sum":{"$sum":"$SQ_FT"},
            }
        },
        {   
            "$project": {
                
                "_id":"$_id",
                "conso_sum": 1,
                "sq_ft_sum":1,
                "count_row":1,
                "sq_ft_season": {"$divide":["$sq_ft_sum","$count_row"]},
            }
        },
        {   
            "$group": {
                    
                    "_id" : {"SECTEUR_ACTIVITE" : "$_id.SECTEUR_ACTIVITE", "SEASON":"$_id.season"},
                    "conso_sum" : { "$sum" : "$conso_sum"} ,
                    "sq_ft_avg" : {"$sum":"$sq_ft_season"}
            }
        },
        {   
            "$project": {
                "_id":0,
                "sub_industry":"$_id.SECTEUR_ACTIVITE",
                "season":"$_id.SEASON",
                "conso_energy": "$conso_sum",
                "sq_ft":"$sq_ft_avg",
                "intensity": {"$divide":["$conso_sum","$sq_ft_avg"]},
            }
        },

        {
            "$sort":{"energy":-1}
        }
    ]           
    requete5_saison = list(coll.aggregate(pipeline,allowDiskUse = True))
    return requete5_saison

In [58]:
%%time
req_intensity_season=query_season(coll_enernoc,cond_summer,cond_autumn,cond_winter,cond_spring)
#resultat de la fonction UDF, décrite dans ce qui suit ==> même résultat que la requête query_season
# {'_id': {'sub_ind': 'Other Light Industrial', 'season': 'summer'}, 'value': {'energy': 308925.04059999035, 'sq_ft': 7394.0, 'count': 27072.0, 'intensity': 41.780503191775814}}
# {'_id': {'sub_ind': 'Other Light Industrial', 'season': 'spring'}, 'value': {'energy': 303362.24989997863, 'sq_ft': 7394.0, 'count': 26496.0, 'intensity': 41.0281647146306}}
# {'_id': {'sub_ind': 'Other Light Industrial', 'season': 'winter'}, 'value': {'energy': 300166.5342999911, 'sq_ft': 7394.0, 'count': 25920.0, 'intensity': 40.59596081958224}}
# {'_id': {'sub_ind': 'Other Light Industrial', 'season': 'autumn'}, 'value': {'energy': 282816.0278999892, 'sq_ft': 7394.0, 'count': 25920.0, 'intensity': 38.24939517175943}}

Wall time: 52.3 s


In [59]:
req_intensity_season

[{'sub_industry': 'Food Processing',
  'season': 'spring',
  'conso_energy': 34181366.2504,
  'sq_ft': 1133956.0,
  'intensity': 30.14346786859455},
 {'sub_industry': 'Other Light Industrial',
  'season': 'spring',
  'conso_energy': 303362.2499,
  'sq_ft': 7394.0,
  'intensity': 41.02816471463348},
 {'sub_industry': 'Grocer/Market',
  'season': 'spring',
  'conso_energy': 11903719.8853,
  'sq_ft': 1045712.0,
  'intensity': 11.38336356979742},
 {'sub_industry': 'Commercial Real Estate',
  'season': 'summer',
  'conso_energy': 6457394.6974,
  'sq_ft': 1385021.0,
  'intensity': 4.662308150851143},
 {'sub_industry': 'Food Processing',
  'season': 'summer',
  'conso_energy': 38970824.8867,
  'sq_ft': 1133956.0,
  'intensity': 34.36714024768157},
 {'sub_industry': 'Commercial Real Estate',
  'season': 'winter',
  'conso_energy': 6351976.8415,
  'sq_ft': 1385021.0,
  'intensity': 4.5861953295293},
 {'sub_industry': 'Manufacturing',
  'season': 'winter',
  'conso_energy': 15358765.598000001,
 

In [32]:
# new_season=db.intensity_industry_season

In [60]:
db.intensity_industry_season.remove()
db.intensity_industry_season.insert(req_intensity_season)

C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e908372131d6bff75fbf440'),
 ObjectId('5e908372131d6bff75fbf441'),
 ObjectId('5e908372131d6bff75fbf442'),
 ObjectId('5e908372131d6bff75fbf443'),
 ObjectId('5e908372131d6bff75fbf444'),
 ObjectId('5e908372131d6bff75fbf445'),
 ObjectId('5e908372131d6bff75fbf446'),
 ObjectId('5e908372131d6bff75fbf447'),
 ObjectId('5e908372131d6bff75fbf448'),
 ObjectId('5e908372131d6bff75fbf449'),
 ObjectId('5e908372131d6bff75fbf44a'),
 ObjectId('5e908372131d6bff75fbf44b'),
 ObjectId('5e908372131d6bff75fbf44c'),
 ObjectId('5e908372131d6bff75fbf44d'),
 ObjectId('5e908372131d6bff75fbf44e'),
 ObjectId('5e908372131d6bff75fbf44f'),
 ObjectId('5e908372131d6bff75fbf450'),
 ObjectId('5e908372131d6bff75fbf451'),
 ObjectId('5e908372131d6bff75fbf452'),
 ObjectId('5e908372131d6bff75fbf453'),
 ObjectId('5e908372131d6bff75fbf454'),
 ObjectId('5e908372131d6bff75fbf455'),
 ObjectId('5e908372131d6bff75fbf456'),
 ObjectId('5e908372131d6bff75fbf457'),
 ObjectId('5e908372131d6bff75fbf458'),
 ObjectId('5e908372131d6b

Requete 5 bis

la même chose que la 5, et par site id (pour le dashboard tableau)

In [61]:
def query_season_site(coll,cond_summer,cond_cond_autumn,cond_winter,cond_spring):

    # order: 1)unwind, 1bis)project1, 2)match (using filters created in 1bis), 3)group, 4)project2
    pipeline = [
        {
            "$unwind":"$ld_data"
        },
        {   
            "$project": {
    #             "_id":"$_id.SUB_INDUSTRY",
                "SUB_INDUSTRY":1,
                "ld_data.value" : 1,
                "SQ_FT":1,
                "SITE_ID":1,
                "LAT":1,
                "LNG":1,
                "month_":{ "$month": "$ld_data.dttm_utc" },
                "day":{ "$dayOfMonth": "$ld_data.dttm_utc" },
                
            }
        },
        {   
            "$group": {
                        "_id" : {"SECTEUR_ACTIVITE" : "$SUB_INDUSTRY", "SITE_ID":"$SITE_ID","LAT":"$LAT","LNG":"$LNG",
                
                        "season":{"$cond":[
                            cond_summer,"summer", {"$cond":[
                                cond_autumn,"autumn", {"$cond":[
                                    cond_winter,"winter", {"$cond":[
                                        cond_spring,"spring","other"
                                                            ]}
                                ]}
                            ]}
                        ]}
                    },
                    "conso_sum" : { "$sum" : "$ld_data.value"} ,
                    "count_row":{"$sum":1},
                    "sq_ft_sum":{"$sum":"$SQ_FT"},
            }
        },
        {   
            "$project": {
                
                "_id":"$_id",
                "conso_sum": 1,
                "sq_ft_sum":1,
                "count_row":1,
                "sq_ft_season": {"$divide":["$sq_ft_sum","$count_row"]},
            }
        },
#         {   
#             "$group": {
                    
#                     "_id" : {"SECTEUR_ACTIVITE" : "$_id.SECTEUR_ACTIVITE", "SEASON":"$_id.season"},
#                     "conso_sum" : { "$sum" : "$conso_sum"} ,
#                     "sq_ft_sum":{"$sum":"$sq_ft_season"}
#             }
#         },
        {   
            "$project": {
                "_id":0,
                "sub_industry":"$_id.SECTEUR_ACTIVITE",
                "season":"$_id.season",
                "site_id":"$_id.SITE_ID",
                "LAT":"$_id.LAT",
                "LNG":"$_id.LNG",
                "conso_energy": "$conso_sum",
                "sq_ft":"$sq_ft_season",
                "intensity": {"$divide":["$conso_sum","$sq_ft_season"]},
            }
        },

        {
            "$sort":{"energy":-1}
        }
    ]           
    req = list(coll.aggregate(pipeline,allowDiskUse = True)) #example test collection
    # requete5_saison = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True)) # collection with all data
    return req

In [62]:
%%time
req_intensity_season_site=query_season_site(coll_enernoc,cond_summer,cond_autumn,cond_winter,cond_spring)

Wall time: 55 s


In [63]:
req_intensity_season_site

[{'sub_industry': 'Primary/Secondary School',
  'season': 'winter',
  'site_id': 99,
  'LAT': 36.57549581,
  'LNG': -86.22741261,
  'conso_energy': 161974.9932,
  'sq_ft': 79503.0,
  'intensity': 2.0373444171918043},
 {'sub_industry': 'Food Processing',
  'season': 'spring',
  'site_id': 744,
  'LAT': 27.84862487,
  'LNG': -82.0480141,
  'conso_energy': 490681.0923,
  'sq_ft': 22383.0,
  'intensity': 21.922043171156684},
 {'sub_industry': 'Grocer/Market',
  'season': 'autumn',
  'site_id': 339,
  'LAT': 34.42182061,
  'LNG': -117.7651881,
  'conso_energy': 634691.3839,
  'sq_ft': 27988.0,
  'intensity': 22.67726825425182},
 {'sub_industry': 'Food Processing',
  'season': 'spring',
  'site_id': 737,
  'LAT': 40.44768449,
  'LNG': -75.99049611,
  'conso_energy': 1271624.6022,
  'sq_ft': 23843.0,
  'intensity': 53.333246747473055},
 {'sub_industry': 'Food Processing',
  'season': 'winter',
  'site_id': 766,
  'LAT': 43.00244098,
  'LNG': -114.1137001,
  'conso_energy': 8032415.3367,
  'sq

In [64]:
db.intensity_industry_season_site.insert(req_intensity_season_site)

C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5e908433131d6bff75fbf468'),
 ObjectId('5e908433131d6bff75fbf469'),
 ObjectId('5e908433131d6bff75fbf46a'),
 ObjectId('5e908433131d6bff75fbf46b'),
 ObjectId('5e908433131d6bff75fbf46c'),
 ObjectId('5e908433131d6bff75fbf46d'),
 ObjectId('5e908433131d6bff75fbf46e'),
 ObjectId('5e908433131d6bff75fbf46f'),
 ObjectId('5e908433131d6bff75fbf470'),
 ObjectId('5e908433131d6bff75fbf471'),
 ObjectId('5e908433131d6bff75fbf472'),
 ObjectId('5e908433131d6bff75fbf473'),
 ObjectId('5e908433131d6bff75fbf474'),
 ObjectId('5e908433131d6bff75fbf475'),
 ObjectId('5e908433131d6bff75fbf476'),
 ObjectId('5e908433131d6bff75fbf477'),
 ObjectId('5e908433131d6bff75fbf478'),
 ObjectId('5e908433131d6bff75fbf479'),
 ObjectId('5e908433131d6bff75fbf47a'),
 ObjectId('5e908433131d6bff75fbf47b'),
 ObjectId('5e908433131d6bff75fbf47c'),
 ObjectId('5e908433131d6bff75fbf47d'),
 ObjectId('5e908433131d6bff75fbf47e'),
 ObjectId('5e908433131d6bff75fbf47f'),
 ObjectId('5e908433131d6bff75fbf480'),
 ObjectId('5e908433131d6b

### Requete 6
•	Répondre à la requête précédente en implémentant un UDF (fonction définie par l'utilisateur) qui calcule directement l'intensité énergétique . Cette requête dépend de la solution choisie 

In [223]:
mongo_client = MongoClient('mongodb://localhost:27017')
db_name='telecom_ds'
# coll_name="data_embedded"
db = mongo_client[db_name]
# coll = db[coll_name]

In [47]:
db.coll_enernoc

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'telecom_ds'), 'coll_enernoc')

### FINAL VERSION Intensity UDF

In [70]:
%%time
# 7min 46s
from bson.code import Code
mapEnergy = Code(
    """
    function () {
                //let mapResult = new Map();
                const mapResult = {};
                //let dicoResult={};
                for (idx=0; idx<this.ld_data.length; idx++){
                    datetmm=new Date(this.ld_data[idx].dttm_utc).toISOString();
                    month=parseInt(datetmm.substring(5,7));
                    day=parseInt(datetmm.substring(8,10));
                    if((month==12 && day>=21)||(month==3 && day<=19)||(month == 1 || month==2)){
                        season='winter';
                    } else if ((month==3 && day>=20)||(month==6 && day<=19)||(month == 4 || month==5)){
                        season='spring';
                    } else if ((month==6 && day>=20)||(month==9 && day<=21)||(month == 7 || month==8)){
                        season='summer';
                    } else if ((month==9 && day>=22)||(month==12 && day<=20)||(month == 10 || month==11)){
                        season='autumn';
                    } else {
                        season='other';
                    }
                    conso=this.ld_data[idx].value;
                    temporary_array=[this.SUB_INDUSTRY,this.SITE_ID,season,conso,this.SQ_FT, 1];
                    key_temp=this.SUB_INDUSTRY+this.SITE_ID+season;
                    value_temp=[conso,this.SQ_FT, 1];
                    
                    if (!mapResult[key_temp]) {
                        mapResult[key_temp]=temporary_array;
                    }
                    else {
                            let e = mapResult[key_temp];
                            e[3] += temporary_array[3];//conso
                            e[4] += temporary_array[4];//sq_ft
                            e[5] += temporary_array[5];//count
                    }
              };
                Object.keys(mapResult).forEach(function(k){
                    v=mapResult[k];
                    emit({sub_industry:v[0],season:v[2]},{consoEnergy:v[3],sq_ft:v[4]/v[5]});
                
                });
            };
            """
)
reduceEnergy= Code("function (key, values) {"
              " calculation = { energy: 0, sq_ft: 0};"
              "  var total = 0;"
              " var count=0;"
              "  for (var i = 0; i < values.length; i++) {"
              "    calculation.energy+=values[i].consoEnergy;"
              "    calculation.sq_ft+=values[i].sq_ft;"
              "  }"
              "  return calculation;"
              "}")

finalizeEnergy = Code("""
            function(key,calculation){
                calculation.intensity=calculation.energy/calculation.sq_ft;
                return calculation;
            };

""")


result=coll_enernoc.map_reduce(mapEnergy,reduceEnergy,"udf_intensity_secteur",finalize=finalizeEnergy)

for doc in result.find().sort("value.intensity",-1):
    print(doc)

{'_id': {'sub_industry': 'Food Processing', 'season': 'summer'}, 'value': {'energy': 38970824.886699244, 'sq_ft': 1133956.0, 'intensity': 34.3671402476809}}
{'_id': {'sub_industry': 'Food Processing', 'season': 'autumn'}, 'value': {'energy': 36114707.08219982, 'sq_ft': 1133956.0, 'intensity': 31.848420117006146}}
{'_id': {'sub_industry': 'Food Processing', 'season': 'spring'}, 'value': {'energy': 34181366.25040034, 'sq_ft': 1133956.0, 'intensity': 30.14346786859485}}
{'_id': {'sub_industry': 'Food Processing', 'season': 'winter'}, 'value': {'energy': 33332818.499200802, 'sq_ft': 1133956.0, 'intensity': 29.39516039352568}}
{'_id': {'sub_industry': 'Grocer/Market', 'season': 'summer'}, 'value': {'energy': 13849428.956600009, 'sq_ft': 1045712.0, 'intensity': 13.244018388045665}}
{'_id': {'sub_industry': 'Manufacturing', 'season': 'summer'}, 'value': {'energy': 19383652.983699895, 'sq_ft': 1584376.0, 'intensity': 12.234250571644544}}
{'_id': {'sub_industry': 'Grocer/Market', 'season': 'spr

### CREATION OF NEW COLLECTION TO STORE UDF FUNCTIONS ONLY

In [71]:
# create collection dedicated only to function
mongo_client = MongoClient('mongodb://localhost:27017')
db_name='telecom_ds'
db = mongo_client[db_name]
# coll_name="energy_function"
coll_fct=db.energy_function

In [73]:
# ===> à modifier
coll_fct.insert(
   {
     "_id" : "mapEnergySesaon" ,
     "value" : mapEnergy
   }
);
coll_fct.insert(
   {
     "_id" : "reduceEnergySesaon" ,
     "value" : reduceEnergy
   }
);
coll_fct.insert(
   {
     "_id" : "finalizeEnergySesaon" ,
     "value" : finalizeEnergy
   }
);

C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


### now we call the function from the db and pass them to map_reduce 

In [ ]:
# coll_fct=db.coll_fct

In [74]:
id_functions=["mapEnergySesaon","reduceEnergySesaon","finalizeEnergySesaon"]
list_functions=[]
for fct in id_functions:
    cursorD=coll_fct.find({"_id":fct},{"value":1,"_id":0})
    list_curs=list(cursorD).copy()
    fct_ernergy=list_curs[0]['value']
    list_functions.append(fct_ernergy)

map_ernergy=list_functions[0]
reduce_ernergy=list_functions[1]
finalize_ernergy=list_functions[2]

In [56]:
%%time
# Wall time: 9min 30s
result=coll_enernoc.map_reduce(map_ernergy,reduce_ernergy,"udf_stored_fct",finalize=finalize_ernergy)

for doc in result.find().sort("value.intensity",-1):
    print(doc)

{'_id': {'sub_ind': 'Manufacturing', 'season': 'summer'}, 'value': {'energy': 19383652.98369794, 'sq_ft': 316875.2, 'count': 135360.0, 'intensity': 61.17125285821654}}
{'_id': {'sub_ind': 'Manufacturing', 'season': 'spring'}, 'value': {'energy': 17974229.845304202, 'sq_ft': 316875.2, 'count': 132480.0, 'intensity': 56.723371994097995}}
{'_id': {'sub_ind': 'Manufacturing', 'season': 'winter'}, 'value': {'energy': 15358765.598003024, 'sq_ft': 316876.2281267602, 'count': 129599.0, 'intensity': 48.469289377741035}}
{'_id': {'sub_ind': 'Manufacturing', 'season': 'autumn'}, 'value': {'energy': 15348844.508704364, 'sq_ft': 317836.3364799452, 'count': 128498.0, 'intensity': 48.29166066628396}}
{'_id': {'sub_ind': 'Other Light Industrial', 'season': 'summer'}, 'value': {'energy': 308925.04059999035, 'sq_ft': 7394.0, 'count': 27072.0, 'intensity': 41.780503191775814}}
{'_id': {'sub_ind': 'Other Light Industrial', 'season': 'spring'}, 'value': {'energy': 303362.24989997863, 'sq_ft': 7394.0, 'coun

In [75]:
map_ernergy #ce qui est à l'intérieur du Code est du javascript

Code("\n    function () {\n                //let mapResult = new Map();\n                const mapResult = {};\n                //let dicoResult={};\n                for (idx=0; idx<this.ld_data.length; idx++){\n                    datetmm=new Date(this.ld_data[idx].dttm_utc).toISOString();\n                    month=parseInt(datetmm.substring(5,7));\n                    day=parseInt(datetmm.substring(8,10));\n                    if((month==12 && day>=21)||(month==3 && day<=19)||(month == 1 || month==2)){\n                        season='winter';\n                    } else if ((month==3 && day>=20)||(month==6 && day<=19)||(month == 4 || month==5)){\n                        season='spring';\n                    } else if ((month==6 && day>=20)||(month==9 && day<=21)||(month == 7 || month==8)){\n                        season='summer';\n                    } else if ((month==9 && day>=22)||(month==12 && day<=20)||(month == 10 || month==11)){\n                        season='autumn';\n  

#### EXAMPLE MAP REDUCE ==> documentation pymongo map reduce


In [ ]:
# mongo_client = MongoClient('mongodb://localhost:27017')
# db_name='telecom_ds'
# db = mongo_client[db_name]

# # db.things.remove()

# db.things.insert({"x": 1, "tags": ["dog", "cat"]})
# db.things.insert({"x": 2, "tags": ["cat"]})
# db.things.insert({"x": 3, "tags": ["mouse", "cat", "dog"]})
# db.things.insert({"x": 4, "tags": []})

# from bson.code import Code
# map = Code("function () {"
#             "  this.tags.forEach(function(z) {"
#             "    emit(z, 1);"
#             "  });"
#             "}")

# reduce = Code("function (key, values) {"
#                "  var total = 0;"
#                "  for (var i = 0; i < values.length; i++) {"
#                "    total += values[i];"
#                "  }"
#                "  return total;"
#                "}")

# result = db.things.map_reduce(map, reduce, "myresults")

# for doc in result.find():
#     print(doc)
    
# list(db.things.find()).copy()



In [ ]:
# mongo_client = MongoClient('mongodb://localhost:27017')
# db_name='telecom_ds'
# coll_name="Enernoc"
# # coll_name="data_test"
# db = mongo_client[db_name]
# coll = db[coll_name]
# # db.loadServerScripts()

### Requete 7 
•	Trouver le jour de consommation le plus élevé par site

In [45]:
# coll_name="data_embedded"
db = mongo_client[db_name]
# coll = db[coll_name]
coll_test=db.data_two_sites

In [86]:
pipeline = [
        {
            "$unwind":"$ld_data"
        },
        {   
            "$project": {
                "SITE_ID":1,
                "ld_data.value" : 1,
                "SQ_FT":1,
                "year_":{ "$year": "$ld_data.dttm_utc" },
                "month_":{ "$month": "$ld_data.dttm_utc" },
                "day_":{ "$dayOfMonth": "$ld_data.dttm_utc" },
            }
        },
            {   
            "$group": {
                        "_id" : {"SITE_ID" : "$SITE_ID", "date":{"$concat":[{"$toString":"$day_"},'-',{"$toString":"$month_"},'-',{"$toString":"$year_"}]}},
                        "conso_sum" : { "$sum" : "$ld_data.value"},
                        "count_row":{"$sum":1},
                        "sq_ft_sum" : { "$sum" : "$SQ_FT"}
            }
        },
        {   
            "$project": {
                "_id":"$_id",
                "conso_sum":1,
                "sq_ft_sum":1,
                "count_row":1,
                "sq_ft_avg":{ "$divide": ["$sq_ft_sum", "$count_row"] },
            }
        },
        {
         
            "$project": {
                "_id":"$_id",
                "conso_sum":1,
                "sq_ft":"$sq_ft_avg",
                "energy": {"$divide": ["$conso_sum", "$sq_ft_avg"]},
            }
        
        },
        {
            "$sort":{"_id.SITE_ID":-1,"energy":-1}
        },
        {
            "$group":{
                "_id":"$_id.SITE_ID",
                "items":{"$push":{"energy":"$energy","date":"$_id.date"}}
            }
        
        },
        {
            "$project":{"items":{"$slice":["$items",1]}}
        },
        {
            "$sort":{"_id":1}
        },
   
    ]           

requete7 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True)) # collection with all data

requete7

[{'_id': 6, 'items': [{'energy': 0.08370001300052002, 'date': '17-1-2012'}]},
 {'_id': 8, 'items': [{'energy': 0.06613526589689381, 'date': '20-3-2012'}]},
 {'_id': 9, 'items': [{'energy': 0.07469394168339039, 'date': '7-7-2012'}]},
 {'_id': 10, 'items': [{'energy': 0.11948207590226433, 'date': '6-7-2012'}]},
 {'_id': 12, 'items': [{'energy': 0.08172921604096513, 'date': '9-7-2012'}]},
 {'_id': 13, 'items': [{'energy': 0.08004157936367011, 'date': '9-7-2012'}]},
 {'_id': 14, 'items': [{'energy': 0.06416891270618003, 'date': '19-1-2012'}]},
 {'_id': 21, 'items': [{'energy': 0.07062621373960117, 'date': '20-3-2012'}]},
 {'_id': 22, 'items': [{'energy': 0.06962984943262189, 'date': '2-10-2012'}]},
 {'_id': 25, 'items': [{'energy': 0.029741948284286467, 'date': '20-3-2012'}]},
 {'_id': 29, 'items': [{'energy': 0.07955099473693969, 'date': '4-1-2012'}]},
 {'_id': 30, 'items': [{'energy': 0.020432648630359082, 'date': '17-1-2012'}]},
 {'_id': 31, 'items': [{'energy': 0.021029830205904612, 'd

In [77]:
requete7_saison #le résultat pour la collection test

[{'_id': 6,
  'items': [{'energy': 0.08370001300052002, 'date': '17-1-2012'},
   {'energy': 0.080208537008147, 'date': '12-1-2012'}]},
 {'_id': 9,
  'items': [{'energy': 0.07469394168339039, 'date': '7-7-2012'},
   {'energy': 0.07024741057726361, 'date': '18-7-2012'}]},
 {'_id': 8,
  'items': [{'energy': 0.06613526589689381, 'date': '20-3-2012'},
   {'energy': 0.06273920926348903, 'date': '26-1-2012'}]},
 {'_id': 10,
  'items': [{'energy': 0.11948207590226433, 'date': '6-7-2012'},
   {'energy': 0.11937079767099955, 'date': '18-7-2012'}]},
 {'_id': 41,
  'items': [{'energy': 0.06709043089677062, 'date': '7-7-2012'},
   {'energy': 0.06309654405546217, 'date': '18-7-2012'}]}]

### Requete 7 bis
•	Trouver le jour de consommation le plus élevé par secteur d'activité

In [65]:
pipeline = [
        {
            "$unwind":"$ld_data"
        },
        {   
            "$project": {
                "SUB_INDUSTRY":1,
                "SITE_ID":1,
                "ld_data.value" : 1,
                "SQ_FT":1,
                "year_":{ "$year": "$ld_data.dttm_utc" },
                "month_":{ "$month": "$ld_data.dttm_utc" },
                "day_":{ "$dayOfMonth": "$ld_data.dttm_utc" },
            }
        },
        {   
            "$group": {
                       "_id" : {"SECTEUR_ACTIVITE" : "$SUB_INDUSTRY","SITE_ID" : "$SITE_ID", "date":{"$concat":[{"$toString":"$day_"},'-',{"$toString":"$month_"},'-',{"$toString":"$year_"}]}},
                        "conso_sum" : { "$sum" : "$ld_data.value"},
                        "count_row":{"$sum":1},
                        "sq_ft_sum" : { "$sum" : "$SQ_FT"}
            }
        },

        {   
            "$project": {
                "conso_sum":1,
                "sq_ft_sum":1,
                "count_row":1,
                "sq_ft_avg":{ "$divide": ["$sq_ft_sum", "$count_row"] },
            }
        },
        {   
            "$group": {
                    
                    "_id" : {"SECTEUR_ACTIVITE" : "$_id.SECTEUR_ACTIVITE", "date":"$_id.date"},
                    "conso_sum" : { "$sum" : "$conso_sum"} ,
                    "sq_ft_sum" : { "$sum" : "$sq_ft_avg"}
            }
        },
        {
         
            "$project": {
                "_id":1,
                "intensity": {"$divide": ["$conso_sum", "$sq_ft_sum"]},
            }
        
        },
     #c'est à ce moment qu'on commence à faire filter en ordre decroissant energy et ne prendre que la premier line par id
        {
            "$sort":{"_id.SECTEUR_ACTIVITE":-1,"intensity":-1}
        },
   
        {
            "$group":{
                "_id":"$_id.SECTEUR_ACTIVITE",
                "items":{"$push":{"intensity":"$intensity","date":"$_id.date"}}
            }
        },
        {
            "$project":{"items":{"$slice":["$items",1]}}
        },
        {
            "$sort":{"_id":1}
        },
    ]           

requete7bis = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True)) #example test collection

requete7bis
# requete7_saison
# https://stackoverflow.com/questions/46111886/mongodb-group-by-with-sort-and-limit-for-each-group

[{'_id': 'Bank/Financial Services',
  'items': [{'energy': 0.03463532695756567, 'date': '19-1-2012'}]},
 {'_id': 'Business Services',
  'items': [{'energy': 0.0623019000903315, 'date': '19-1-2012'}]},
 {'_id': 'Commercial Real Estate',
  'items': [{'energy': 0.06860885972126055, 'date': '4-1-2012'}]},
 {'_id': 'Corporate Office',
  'items': [{'energy': 0.0706590723985661, 'date': '7-7-2012'}]},
 {'_id': 'Food Processing',
  'items': [{'energy': 0.4317113534387577, 'date': '20-11-2012'}]},
 {'_id': 'Grocer/Market',
  'items': [{'energy': 0.1567311299860765, 'date': '11-8-2012'}]},
 {'_id': 'Manufacturing',
  'items': [{'energy': 0.16295831664958318, 'date': '26-7-2012'}]},
 {'_id': 'Other Light Industrial',
  'items': [{'energy': 0.728700838517717, 'date': '1-8-2012'}]},
 {'_id': 'Primary/Secondary School',
  'items': [{'energy': 0.03817099686245235, 'date': '5-9-2012'}]},
 {'_id': 'Shopping Center/Shopping Mall',
  'items': [{'energy': 0.06011499840028655, 'date': '6-9-2012'}]}]

### Requete 8
•	Trouver les 3 premiers SITE_ID qui consomment le plus par secteur d'activité en terme d'intensité énergétique

In [71]:
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "SUB_INDUSTRY":1,
            "ld_data.value" : 1,
            "SITE_ID":1,
            "SQ_FT":1
        }
    },
    {   
        "$group": {
                    "_id" : {"SUB_INDUSTRY":"$SUB_INDUSTRY","SITE_ID" : "$SITE_ID"},
                    "conso_sum" : { "$sum" : "$ld_data.value"},
                    "count_row":{"$sum":1},
                    "sq_ft_sum" : { "$sum" : "$SQ_FT"}
                    
        }
    },
    {
        "$project":{
            "conso_sum":1,
            "sq_ft_sum":1,
            "sq_ft_avg":{ "$divide": ["$sq_ft_sum", "$count_row"] }
        }
    },
    {
        "$project":{
            "_id":1,
            "intensity":{ "$divide": ["$conso_sum", "$sq_ft_avg"] }
        }
    },
    {
        "$sort":{"_id.SUB_INDUSTRY":-1,"intensity":-1}
    },
    {
        "$group":{
            "_id":"$_id.SUB_INDUSTRY",
            "items":{"$push":{"intensity":"$intensity","SITE_ID":"$_id.SITE_ID"}}
            }
    },
    {
        "$sort":{"intensity":-1}
    },
    {
        "$project":{"items":{"$slice":["$items",3]}}
    },
    {
        "$sort":{"_id":1}
    },
]

requete8bis = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

requete8bis

[{'_id': 'Bank/Financial Services',
  'items': [{'intensity': 19.516753960016345, 'SITE_ID': 22},
   {'intensity': 3.4332225718354055, 'SITE_ID': 30}]},
 {'_id': 'Business Services',
  'items': [{'intensity': 23.18115327025297, 'SITE_ID': 12},
   {'intensity': 13.713538386305588, 'SITE_ID': 14},
   {'intensity': 10.983646788954479, 'SITE_ID': 36}]},
 {'_id': 'Commercial Real Estate',
  'items': [{'intensity': 22.702330578917067, 'SITE_ID': 13},
   {'intensity': 21.258474352113236, 'SITE_ID': 56},
   {'intensity': 20.698212106138424, 'SITE_ID': 65}]},
 {'_id': 'Corporate Office',
  'items': [{'intensity': 17.395732770038954, 'SITE_ID': 9},
   {'intensity': 15.624829940904393, 'SITE_ID': 41}]},
 {'_id': 'Food Processing',
  'items': [{'intensity': 2816.4419260256095, 'SITE_ID': 766},
   {'intensity': 938.4497248584663, 'SITE_ID': 745},
   {'intensity': 857.2220329489292, 'SITE_ID': 771}]},
 {'_id': 'Grocer/Market',
  'items': [{'intensity': 112.61745789353958, 'SITE_ID': 512},
   {'inten

In [ ]:
[{'_id': 'Bank/Financial Services',
  'items': [{'intensity': 19.516753960016345, 'SITE_ID': 22},
   {'intensity': 3.4332225718354055, 'SITE_ID': 30}]},
 {'_id': 'Business Services',
  'items': [{'intensity': 23.18115327025297, 'SITE_ID': 12},
   {'intensity': 13.713538386305588, 'SITE_ID': 14},
   {'intensity': 10.983646788954479, 'SITE_ID': 36}]},
 {'_id': 'Commercial Real Estate',
  'items': [{'intensity': 22.702330578917067, 'SITE_ID': 13},
   {'intensity': 21.258474352113236, 'SITE_ID': 56},
   {'intensity': 20.698212106138424, 'SITE_ID': 65}]},
 {'_id': 'Corporate Office',
  'items': [{'intensity': 17.395732770038954, 'SITE_ID': 9},
   {'intensity': 15.624829940904393, 'SITE_ID': 41}]},
 {'_id': 'Food Processing',
  'items': [{'intensity': 2816.4419260256095, 'SITE_ID': 766},
   {'intensity': 938.4497248584663, 'SITE_ID': 745},
   {'intensity': 857.2220329489292, 'SITE_ID': 771}]},
 {'_id': 'Grocer/Market',
  'items': [{'intensity': 112.61745789353958, 'SITE_ID': 512},
   {'intensity': 107.96799302487793, 'SITE_ID': 475},
   {'intensity': 96.03311316635701, 'SITE_ID': 339}]},
 {'_id': 'Manufacturing',
  'items': [{'intensity': 59.53118059423518, 'SITE_ID': 690},
   {'intensity': 56.95207206094506, 'SITE_ID': 755},
   {'intensity': 43.89479107783982, 'SITE_ID': 832}]},
 {'_id': 'Other Light Industrial',
  'items': [{'intensity': 161.65402389775494, 'SITE_ID': 765}]},
 {'_id': 'Primary/Secondary School',
  'items': [{'intensity': 13.230978375325234, 'SITE_ID': 101},
   {'intensity': 12.881758639704465, 'SITE_ID': 224},
   {'intensity': 12.090219326666736, 'SITE_ID': 109}]},
 {'_id': 'Shopping Center/Shopping Mall',
  'items': [{'intensity': 57.44907461322704, 'SITE_ID': 44},
   {'intensity': 56.5699979321114, 'SITE_ID': 51},
   {'intensity': 45.855038440706274, 'SITE_ID': 55}]}]

In [68]:
db.intensity_important_sites.insert(requete8)

C:\Users\Hanane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


['Bank/Financial Services',
 'Business Services',
 'Commercial Real Estate',
 'Corporate Office',
 'Food Processing',
 'Grocer/Market',
 'Manufacturing',
 'Other Light Industrial',
 'Primary/Secondary School',
 'Shopping Center/Shopping Mall']

Trouver le client qui consomme le plus par secteur d'activité et par année

In [102]:
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "SUB_INDUSTRY":1,
            "ld_data.value" : 1,
            "SITE_ID":1,
            "SQ_FT":1,
            "year_":{ "$year": "$ld_data.dttm_utc" },
        }
    },
    {   
        "$group": {
                    "_id" : {"SUB_INDUSTRY":"$SUB_INDUSTRY","SITE_ID" : "$SITE_ID","year":"$year_"},
                    "conso_sum" : { "$sum" : "$ld_data.value"},
                    "count_row":{"$sum":1},
                    "sq_ft_sum" : { "$sum" : "$SQ_FT"}
                    
        }
    },
    {
        "$project":{
            "conso_sum":1,
            "sq_ft_sum":1,
            "sq_ft_avg":{ "$divide": ["$sq_ft_sum", "$count_row"] }
        }
    },
    {
        "$project":{
            "energy":{ "$divide": ["$conso_sum", "$sq_ft_avg"] }
        }
    },
    {
        "$sort":{"energy":-1}
    },
#          c'est à ce moment qu'on commence à faire filter en ordre decroissant energy et ne prendre que la premier line par id
    {
        "$sort":{"_id.SUB_INDUSTRY":-1,"_id.SITE_ID":1,"_id.year":1,"energy":-1}
    },
    {
        "$group":{
            "_id":{"SUB_INDUSTRY":"$_id.SUB_INDUSTRY","year":"$_id.year"},
            "items":{"$push":{"energy":"$energy","SITE_ID":"$_id.SITE_ID"}}
            }
    },
    {
        "$project":{"items":{"$slice":["$items",1]}}
    },
    {
        "$sort":{"_id.SUB_INUDSTRY":-1}
    },
]

# requete8bis = list(coll_test.aggregate(pipeline,allowDiskUse = True)) #example test collection
requete8 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True)) # collection with all data
requete8
# requete8bis

[{'_id': {'SUB_INDUSTRY': 'Commercial Real Estate', 'year': 2012},
  'items': [{'energy': 22.70215611336207, 'SITE_ID': 13}]},
 {'_id': {'SUB_INDUSTRY': 'Shopping Center/Shopping Mall', 'year': 2012},
  'items': [{'energy': 20.266945517915953, 'SITE_ID': 6}]},
 {'_id': {'SUB_INDUSTRY': 'Corporate Office', 'year': 2013},
  'items': [{'energy': 0.00015182150867666154, 'SITE_ID': 9}]},
 {'_id': {'SUB_INDUSTRY': 'Manufacturing', 'year': 2013},
  'items': [{'energy': 5.615881850014431e-05, 'SITE_ID': 690}]},
 {'_id': {'SUB_INDUSTRY': 'Grocer/Market', 'year': 2012},
  'items': [{'energy': 32.414488477433615, 'SITE_ID': 281}]},
 {'_id': {'SUB_INDUSTRY': 'Corporate Office', 'year': 2012},
  'items': [{'energy': 17.39558094853028, 'SITE_ID': 9}]},
 {'_id': {'SUB_INDUSTRY': 'Primary/Secondary School', 'year': 2012},
  'items': [{'energy': 8.916678577801381, 'SITE_ID': 88}]},
 {'_id': {'SUB_INDUSTRY': 'Primary/Secondary School', 'year': 2013},
  'items': [{'energy': 5.039936271906532e-05, 'SITE_I

### Requete 9
•	Trouver la consommation par jour de semaines (Lundi, Mardi,....), tout site confondu. Trier par ordre decroissant

In [105]:
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "ld_data.value" : 1,
            "SITE_ID":1,
            "SQ_FT":1,
            "dayOfWeek":{ "$dayOfWeek": "$ld_data.dttm_utc" },
        }
    },
    {   
        "$group": {
                    "_id" : {"dayOfWeek":"$dayOfWeek"},
                    "conso_sum" : { "$sum" : "$ld_data.value"},
        }
    },
    {
        "$sort":{"conso_sum":-1}
    }
]

requete9 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

requete9

[{'_id': {'dayOfWeek': 4}, 'conso_sum': 83761614.602},
 {'_id': {'dayOfWeek': 5}, 'conso_sum': 83684628.4371},
 {'_id': {'dayOfWeek': 3}, 'conso_sum': 83055248.9656},
 {'_id': {'dayOfWeek': 6}, 'conso_sum': 80018184.7589},
 {'_id': {'dayOfWeek': 2}, 'conso_sum': 73687905.8993},
 {'_id': {'dayOfWeek': 7}, 'conso_sum': 64225018.494},
 {'_id': {'dayOfWeek': 1}, 'conso_sum': 56357347.3506}]

### Requete 10
•	Trouver la consommation par jour de semaines (Lundi, Mardi,....) par secteur d'activité

In [108]:
pipeline = [
    {
        "$unwind":"$ld_data"
    },
    {   
        "$project": {
            "SUB_INDUSTRY":1,
            "ld_data.value" : 1,
            "SQ_FT":1,
            "dayOfWeek":{ "$dayOfWeek": "$ld_data.dttm_utc" },
        }
    },
    {   
        "$group": {
                    "_id" : {"SUB_INDUSTRY":"$SUB_INDUSTRY","dayOfWeek":"$dayOfWeek"},
                     "conso_sum" : { "$sum" : "$ld_data.value"},
        }
    },
    {
        "$sort":{"_id.SUB_INDUSTRY":1,"conso_sum":-1}
    }
]

requete10 = list(coll_enernoc.aggregate(pipeline,allowDiskUse = True))

requete10

[{'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 6},
  'conso_sum': 1272460.2211},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 4},
  'conso_sum': 1270361.0034},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 5},
  'conso_sum': 1268986.2387},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 3},
  'conso_sum': 1267121.3335},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 2},
  'conso_sum': 1178730.0135},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 7},
  'conso_sum': 918120.3282},
 {'_id': {'SUB_INDUSTRY': 'Bank/Financial Services', 'dayOfWeek': 1},
  'conso_sum': 737739.1706},
 {'_id': {'SUB_INDUSTRY': 'Business Services', 'dayOfWeek': 4},
  'conso_sum': 4541851.4154},
 {'_id': {'SUB_INDUSTRY': 'Business Services', 'dayOfWeek': 3},
  'conso_sum': 4534483.9484},
 {'_id': {'SUB_INDUSTRY': 'Business Services', 'dayOfWeek': 5},
  'conso_sum': 4519437.1022},
 {'_id': {'SUB_INDUS